<center><h1><b>ĐỒ ÁN: EEDI - MINING MISCONCEPTIONS IN MATHEMATICS</b></h1><center>

<center><h2><u>Môn:</u> Học sâu cho Khoa học dữ liệu - <u>Lớp:</u> 21_21</h2></center>

<h3><u>Giáo viên hướng dẫn:</u><h3>

- TS. Nguyễn Tiến Huy
- TS. Lê Thanh Tùng
- ThS. Nguyễn Trần Duy Minh  

<h3><u>Nhóm thực hiện:</u> Nhóm 1</h3>

| Họ và tên | MSSV |
| :--- | :---:|
| Phạm Nhật Duy | 21120058 |
| Nguyễn Trúc Nguyên | 21120102 |
| Trương Công Trung | 21120158 |
| Lê Trần Minh Khuê | 21120279 |

# **Tổng quan**
<u>Trong phạm vi đồ án này</u>, nhóm chúng em sẽ phát triển một mô hình học máy cung cấp khả năng phân tích và dự đoán mối liên hệ giữa *các ngộ nhận* với *các câu trả lời không chính xác* trong các câu hỏi trắc nghiệm.

<u>Mục tiêu:</u> Đối với mỗi câu hỏi trắc nghiệm, mô hình sẽ

- Xác định được câu trả lời đúng,
- Xác định được những lầm tưởng/ ngộ nhận có thể dẫn đến việc chọn đáp án sai.

<u>Trong phạm vi file này</u>, nhóm sẽ tiến hành tiền xử lý cơ bản trước khi đưa vào mô hình.



# **Tiền xử lý dữ liệu**

## 0. Gọi các thư viện cần thiết

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

## 1. Đọc dữ liệu
Đọc dữ liệu đã được tải về từ cuộc thi



In [ ]:
# Đọc dữ liệu từ file train.csv
data = pd.read_csv('./data/train_fillna.csv')

if not data.empty:
    print("Dữ liệu huấn luyện đã được đọc thành công.")
else:
    print("Dữ liệu huấn luyện trống.")

# Đọc dữ liệu từ file test.csv
test = pd.read_csv('./data/test.csv')

if not test.empty:
    print("Dữ liệu kiểm tra đã được đọc thành công.")
else:
    print("Dữ liệu kiểm tra trống.")

# Đọc dữ liệu từ file misconception_mapping.csv
mis_map = pd.read_csv('./data/misconception_mapping.csv')
mis_map = mis_map.astype(object)
if not mis_map.empty:
    print("Dữ liệu misconception đã được đọc thành công.")
else:
    print("Dữ liệu misconception trống.")

Dữ liệu huấn luyện đã được đọc thành công.
Dữ liệu kiểm tra đã được đọc thành công.
Dữ liệu misconception đã được đọc thành công.


## 2. Tiền xử lý dữ liệu các cột "Text"

Trước tiên, in một mẫu dữ liệu các cột text để xem xét

In [ ]:
columns_to_clean = ['QuestionText', 'AnswerAText','AnswerBText','AnswerCText','AnswerDText']
sample = data[columns_to_clean].head()
sample

,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText
0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets need to go to make the answer equal \( 13 \) ?,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets
1,"Simplify the following, if possible: \( \frac{m^{2}+2 m-3}{m-3} \)",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify
2,"Tom and Katie are discussing the \( 5 \) plants with these heights:\n\( 24 \mathrm{~cm}, 17 \mathrm{~cm}, 42 \mathrm{~cm}, 26 \mathrm{~cm}, 13 \mathrm{~cm} \)\nTom says if all the plants were cut in half, the range wouldn't change.\nKatie says if all the plants grew by \( 3 \mathrm{~cm} \) each, the range wouldn't change.\nWho do you agree with?",Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct
3,The angles highlighted on this rectangle with different length sides can never be... ![A rectangle with the diagonals drawn in. The angle on the right hand side at the centre is highlighted in red and the angle at the bottom at the centre is highlighted in yellow.](),acute,obtuse,\( 90^{\circ} \),Not enough information
4,The equation \( f=3 r^{2}+3 \) is used to find values in the table below. What is the value covered by the star? \begin{tabular}{|c|c|c|c|c|}\n\hline\( r \) & \( 1 \) & \( 2 \) & \( 3 \) & \( 4 \) \\\n\hline\( f \) & \( 6 \) & \( 15 \) & \( \color{gold}\bigstar \) & \\\n\hline\n\end{tabular},\( 30 \),\( 27 \),\( 51 \),\( 24 \)


Nhóm nhận thấy trong dữ liệu tồn tại các phần không cần thiết như những ký tự xuống dòng, khoảng trắng thừa, link,... nên sẽ tiến hành xử lý những ký tự này.

In [ ]:
# Làm sạch nhưng bảo toàn công thức toán học
def clean_with_latex_preservation(x):
    # Loại bỏ xuống dòng và khoảng trắng thừa, nhưng bảo toàn LaTeX
    x = re.sub("http\w+", " ", x)            # Xóa link
    x = re.sub(r"[\n\t\r]+", " ", x)         # Xóa xuống dòng, tab
    x = re.sub(r"[ ]{2,}", " ", x)           # Thay khoảng trắng liên tiếp
    x = re.sub(r"(?<!\\)\\\s+", r"\\", x)    # Bảo toàn ký tự \ trong LaTeX
    x = x.strip()                            # Xóa khoảng trắng thừa đầu/cuối
    return x


# Áp dụng hàm làm sạch cho các cột văn bản, giữ định dạng LaTeX
for column in columns_to_clean:
    data[column] = data[column].apply(clean_with_latex_preservation)

# Kiểm tra kết quả sau khi xử lý Markdown
cleaned_sample = data[columns_to_clean].head()
cleaned_sample

<>:4: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\w'
C:\Users\letra\AppData\Local\Temp\ipykernel_10532\307230320.py:4: SyntaxWarning: invalid escape sequence '\w'
  x = re.sub("http\w+", " ", x)            # Xóa link


,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText
0,\[ 3 \times 2+4-5 \] Where do the brackets need to go to make the answer equal \( 13 \) ?,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets
1,"Simplify the following, if possible: \( \frac{m^{2}+2 m-3}{m-3} \)",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify
2,"Tom and Katie are discussing the \( 5 \) plants with these heights: \( 24 \mathrm{~cm}, 17 \mathrm{~cm}, 42 \mathrm{~cm}, 26 \mathrm{~cm}, 13 \mathrm{~cm} \) Tom says if all the plants were cut in half, the range wouldn't change. Katie says if all the plants grew by \( 3 \mathrm{~cm} \) each, the range wouldn't change. Who do you agree with?",Only Tom,Only Katie,Both Tom and Katie,Neither is correct
3,The angles highlighted on this rectangle with different length sides can never be... ![A rectangle with the diagonals drawn in. The angle on the right hand side at the centre is highlighted in red and the angle at the bottom at the centre is highlighted in yellow.](),acute,obtuse,\( 90^{\circ} \),Not enough information
4,The equation \( f=3 r^{2}+3 \) is used to find values in the table below. What is the value covered by the star? \begin{tabular}{|c|c|c|c|c|} \hline\( r \) & \( 1 \) & \( 2 \) & \( 3 \) & \( 4 \) \\ \hline\( f \) & \( 6 \) & \( 15 \) & \( \color{gold}\bigstar \) & \\ \hline \end{tabular},\( 30 \),\( 27 \),\( 51 \),\( 24 \)


Sau khi xử lý, các ký tự không cần thiết đã được loại bỏ. Tuy nhiên, nhóm nhận thấy các phần markdown đang có định dạng không tốt, ví dụ như "$\texttt{![text]()}$" $⇒$ cần xử lý loại bỏ.

In [ ]:
# Hàm xử lý Markdown ![...]()
def extract_markdown_text(x):
    # Tìm và thay thế tất cả mẫu Markdown bằng nội dung trong []
    processed_text = re.sub(r'\!\[([^\]]*)\]\([^\)]*\)', r'\1', x)
    return processed_text

# Áp dụng hàm trên các cột văn bản cần làm sạch
for column in columns_to_clean:
    data[column] = data[column].apply(extract_markdown_text)


# Kiểm tra kết quả sau khi xử lý Markdown
markdown_cleaned_sample = data[columns_to_clean].head()
markdown_cleaned_sample

,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText
0,\[ 3 \times 2+4-5 \] Where do the brackets need to go to make the answer equal \( 13 \) ?,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets
1,"Simplify the following, if possible: \( \frac{m^{2}+2 m-3}{m-3} \)",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify
2,"Tom and Katie are discussing the \( 5 \) plants with these heights: \( 24 \mathrm{~cm}, 17 \mathrm{~cm}, 42 \mathrm{~cm}, 26 \mathrm{~cm}, 13 \mathrm{~cm} \) Tom says if all the plants were cut in half, the range wouldn't change. Katie says if all the plants grew by \( 3 \mathrm{~cm} \) each, the range wouldn't change. Who do you agree with?",Only Tom,Only Katie,Both Tom and Katie,Neither is correct
3,The angles highlighted on this rectangle with different length sides can never be... A rectangle with the diagonals drawn in. The angle on the right hand side at the centre is highlighted in red and the angle at the bottom at the centre is highlighted in yellow.,acute,obtuse,\( 90^{\circ} \),Not enough information
4,The equation \( f=3 r^{2}+3 \) is used to find values in the table below. What is the value covered by the star? \begin{tabular}{|c|c|c|c|c|} \hline\( r \) & \( 1 \) & \( 2 \) & \( 3 \) & \( 4 \) \\ \hline\( f \) & \( 6 \) & \( 15 \) & \( \color{gold}\bigstar \) & \\ \hline \end{tabular},\( 30 \),\( 27 \),\( 51 \),\( 24 \)


Bước đầu đã xử lý làm sạch được dữ liệu. Xuất ra file để sử dụng ở các bước sau.

> Thêm khối trích dẫn



In [ ]:
data.to_csv('./data/train_cleaned.csv', index=False)